In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import numpy as np
import pandas as pd
tickers = pd.read_csv("data/stock_etf_tickers.csv")
# tickers = list(tickers.Ticker)
tickers = ['MSFT', "AAPL", "NVDA", "AMZN"]
# tickers = ['XOM']

In [ ]:
from src.DataCollector import DataCollector
from src.Statistics import Statistics
from src.Portfolio import Portfolio


dc = DataCollector(tickers, "2016-01-01", "2017-12-31")
rt = dc.get_returns()
rt 

[*********************100%***********************]  4 of 4 completed


Ticker,AAPL,AMZN,MSFT,NVDA
Date,,,,
2016-01-05,-0.025059,-0.005024,0.004562,0.016064
2016-01-06,-0.019570,-0.001799,-0.018165,-0.041350
2016-01-07,-0.042204,-0.039058,-0.034783,-0.039645
2016-01-08,0.005288,-0.001464,0.003067,-0.021466
2016-01-11,0.016192,0.017610,-0.000573,0.001688
...,...,...,...,...
2017-12-22,0.000000,-0.005448,0.000117,-0.003165
2017-12-26,-0.025370,0.007190,-0.001286,0.011113
2017-12-27,0.000176,0.004674,0.003630,-0.001368


In [30]:
rt.shape

(502, 4)

In [33]:
months = rt.resample("MS").last().index
months

DatetimeIndex(['2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
               '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01',
               '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',
               '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01',
               '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01'],
              dtype='datetime64[ns]', name='Date', freq='MS')

In [ ]:
stats = Statistics()
port_rt = []

for i in range(12, len(months) - 1):
    beg = months[i - 12]
    end = months[i]
    
    train_rt = rt[(rt.index >= beg) & (rt.index < end)]
    stats.set_rets(train_rt)
    mu = stats.get_mu()
    cov = stats.get_cov()
    
    portfolio = Portfolio(mu, cov)

    # weights at the start of rebalancing period
    init_weights = portfolio.get_MVP()

    # out-sample data
    test_rt = rt[(rt.index >= end) & (rt.index < months[i + 1])]
    gross_rt = (1 + test_rt)
    securities_n = test_rt.shape[1]

    # calculate rolling weights (no artificial daily rebalancing)
    rolling_weights = init_weights.T * gross_rt.cumprod()
    rolling_weights = rolling_weights.shift(1)
    rolling_weights.iloc[0] = init_weights.T
    
    stock_rt = rolling_weights * test_rt
    period_rt = stock_rt @ np.ones(securities_n)
    port_rt.append(period_rt)

port_rt = (pd.concat(port_rt, axis = 0).sort_index().astype(float))
port_rt
# MISSING ONE MONTH. WHY??



    

Date
2017-01-03    0.003950
2017-01-04   -0.001088
2017-01-05    0.006546
2017-01-06    0.011742
2017-01-09    0.003546
                ...   
2017-11-24   -0.001941
2017-11-27    0.002658
2017-11-28    0.007184
2017-11-29   -0.017206
2017-11-30    0.010935
Length: 231, dtype: float64

In [4]:
stats = Statistics(rt)
mu = stats.get_mu()
cov = stats.get_cov()
cov, mu

(array([[1.23488514e-04, 7.49073178e-05, 4.53916680e-05, 9.97277804e-05],
        [7.49073178e-05, 1.75081436e-04, 7.67230560e-05, 9.96229457e-05],
        [4.53916680e-05, 7.67230560e-05, 8.71717579e-05, 7.88134930e-05],
        [9.97277804e-05, 9.96229457e-05, 7.88134930e-05, 6.40931320e-04]]),
 array([[0.00163171],
        [0.00184352],
        [0.00138252],
        [0.00289392]]))

In [60]:
import numpy as np

# CONSTRAINED
mu_p = 0.002
N = mu.shape[0]
ones = np.ones((N, 1))
onesT = np.ones((1, N))
muT = mu.reshape(1, -1)
cov_inv = np.linalg.inv(cov)
ones, onesT

A = onesT @ cov_inv @ ones
B = muT @ cov_inv @ ones
C = muT @ cov_inv @ mu
delta = A * C - B ** 2
lambda1 = (C - B * mu_p) / delta
lambda2 = (A * mu_p - B) / delta
w = lambda1 * cov_inv @ ones + lambda2 * cov_inv @ mu
w


array([[0.44846811],
       [0.10134179],
       [0.29325328],
       [0.15693682]])

In [23]:
# expected return of unconstrained
print(f"Expected return: {w.T @ mu}")
# expected variance of unconstrined
var = w.T @ cov @ w
print(f"Expected sigma ^ 2: {np.sqrt(var)}")



Expected return: [[0.00142284]]
Expected sigma ^ 2: [[0.00849488]]


In [26]:
np.dot(w.T, mu)

array([[0.00142284]])

In [37]:
ff = pd.read_csv("data/ff_daily.csv")
ff['Date'] = pd.to_datetime(ff['Date'], format = "%Y%m%d")
ff.set_index('Date', inplace = True)
common = rt.index.intersection(ff.index)
ff = ff.loc[common]
mkt = ff['Mkt-RF'] / 100
mkt

Date
2017-01-04    0.0079
2017-01-05   -0.0021
2017-01-06    0.0029
2017-01-09   -0.0037
2017-01-10    0.0016
               ...  
2017-12-22   -0.0007
2017-12-26   -0.0007
2017-12-27    0.0005
2017-12-28    0.0022
2017-12-29   -0.0057
Name: Mkt-RF, Length: 250, dtype: float64

In [45]:
import statsmodels.api as sm

alphas = {}
betas = {}
resvars = {}

for stock in tickers:
    y = rt[stock]
    Xdesign = sm.add_constant(mkt)
    model = sm.OLS(y, Xdesign).fit()

    alphas[stock]  = model.params["const"]
    betas[stock]   = model.params["Mkt-RF"]
    resvars[stock] = model.resid.var(ddof=0)

summary = pd.DataFrame({
    "alpha"     : alphas,
    "beta"      : betas,
    "resid_var" : resvars
}).T

summary

,MSFT,AAPL,NVDA,AMZN
alpha,0.000507,0.000723,0.001320,0.000973
beta,1.157999,1.201533,2.082481,1.151410
resid_var,0.000059,0.000093,0.000549,0.000147


In [104]:
print(rt.shape)
print(rt.index[:3], "← index")
print(rt.columns[:3], "← columns")

market = mkt.to_numpy()
x = rt.to_numpy().T
m_bar = market.mean()
x_bar = x.mean(axis = 1, keepdims = True)
x_bar

N, T = x.shape
assert len(tickers) == N
print(tickers, rt.columns.to_list())

m_c = market - m_bar
x_c = x - x_bar

beta_hat = (x_c @ m_c) / (m_c @ m_c)
beta_hat

alpha_hat = x_bar.flatten() - beta_hat * m_bar
alpha_hat

eps = x - alpha_hat[:, None] - beta_hat[:, None] * market
resid_var  = (eps ** 2).mean(axis=1)
resid_var

results = pd.DataFrame(
    {"alpha": alpha_hat,
     "beta":  beta_hat,
     "resid_var": resid_var},
    index=rt.columns.to_list()
)
results

(250, 4)
DatetimeIndex(['2017-01-04', '2017-01-05', '2017-01-06'], dtype='datetime64[ns]', name='Date', freq=None) ← index
Index(['AAPL', 'AMZN', 'MSFT'], dtype='object', name='Ticker') ← columns
['MSFT', 'AAPL', 'NVDA', 'AMZN'] ['AAPL', 'AMZN', 'MSFT', 'NVDA']


,alpha,beta,resid_var
AAPL,0.000723,1.201533,0.000093
AMZN,0.000973,1.151410,0.000147
MSFT,0.000507,1.157999,0.000059
NVDA,0.001320,2.082481,0.000549


In [49]:
import numpy as np

T = 10
N = 4

market = 0.15 * np.random.randn(T)

true_betas = np.array([1.0, 0.8, 1.2, 0.5])
true_alphas = np.array([0.001, -0.0005, 0.0007, 0.0])
res_sd = np.array([0.10, 0.08, 0.12, 0.09])

X = true_alphas[:, None] + true_betas[:, None] * market + res_sd[:, None] * np.random.randn(N, T)
X

array([[-0.0979157 , -0.02554747,  0.12066443, -0.11500772, -0.04471057,
         0.35143176,  0.0890803 ,  0.0430127 ,  0.01592689,  0.3522757 ],
       [-0.31609393, -0.06802273,  0.10060396, -0.00764949, -0.10444228,
         0.30442484,  0.03838578,  0.08427627,  0.06994382,  0.10384034],
       [-0.48921728, -0.35371742,  0.21130517, -0.17983523, -0.2910925 ,
         0.42164161,  0.05676357,  0.03716657,  0.16461968,  0.29032658],
       [-0.25466967,  0.08526089,  0.14645724, -0.02747931, -0.16622787,
         0.32256134,  0.0922575 ,  0.13252063, -0.05082654,  0.09322504]])